In [12]:
%load_ext autoreload
%autoreload 2

import torch 
import os
import glob
from matplotlib import pyplot as plt

import matplotlib.animation as animation
import os
import torch
import numpy as np
import pandas as pd
# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')


from distributions import ZeroInflatedDist, QuantizedNormal
from torch_models import MixtureOfTruncNormModel, SpatialWaves
from torch_training import train_epoch
from torch_perturb.torch_pert_topk import PerturbedBrokenTopK
from torch_perturb.perturbations import perturbed
from metrics import top_k_onehot_indicator
import time
from torch_training import train_epoch_largesynth
from torch_models import torch_bpr_uncurried, deterministic_bpr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
seed=360
num_components=2
init_idx=10
perturbed_noise=0.05
step_size=0.1
threshold = 1
bpr_weight=30
nll_weight=0

In [10]:

# total timepoints
T= 500
S=7
K=5

dist_S = [QuantizedNormal(10, 0.3),
        QuantizedNormal(20, 0.3),
        QuantizedNormal(30, 0.3),
        QuantizedNormal(40, 0.3),
        QuantizedNormal(50, 0.3),
        QuantizedNormal(60, 0.3),
        QuantizedNormal(100, 0.3)]
train_y_TS = np.zeros((T, S))
for s, dist in enumerate(dist_S):
    random_state = np.random.RandomState(10000 * seed + s*123456)
    train_y_TS[:, s] = dist.rvs(size=T, random_state=random_state)

model = MixtureOfTruncNormModel(num_components=num_components, S=S, low=0, high=150)
if init_idx is not None:
    # Reproducibly, randomly generate some numbers using a numpy rng
    init_rng = np.random.RandomState(1989)
    # generate 20 sets of 2 floats between 0.5 and 60
    all_means = init_rng.uniform(8, 102, (20, num_components))
    # generate 20 sets of 2 floats between 0.25 and 6
    all_scales = init_rng.uniform(0.25, 8, (20, num_components))
    # generate 20 lists length S containing lists length 2 which sum to 1
    all_mix_weights = init_rng.dirichlet([0.5]*num_components, (20,S))

    softinv_means = torch.tensor(all_means[init_idx]) + torch.log(-torch.expm1(torch.tensor(-all_means[init_idx])))
    softinv_scales = torch.tensor(all_scales[init_idx]) - 0.2 + torch.log(-torch.expm1(torch.tensor(-all_scales[init_idx] + 0.2)))
    mix_weights = torch.log(1e-13 + torch.tensor(all_mix_weights[init_idx]))
    model.update_params(torch.cat([softinv_means, softinv_scales, mix_weights.view(-1)]))

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=step_size)

M_score_func =  100
M_action = 100
train_T = train_y_TS.shape[0]
perturbed_top_K_func = PerturbedBrokenTopK(k=K, sigma=perturbed_noise)
losses, bprs, nlls = [], [], []
for epoch in range(100):
    print(f'EPOCH: {epoch}')
    loss, bpr, nll, model = train_epoch(model, optimizer, K, threshold, train_T, M_score_func, M_action, train_y_TS, perturbed_top_K_func, bpr_weight, nll_weight)
    losses.append(loss)
    bprs.append(bpr)
    nlls.append(nll)

EPOCH: 0
det bpr: 0.8489137885029586
Pert bpr: 0.8022005155574058
nll: 14.704272689897223
Loss: -24.066015466722174
EPOCH: 1
det bpr: 0.8618141449301174
Pert bpr: 0.803852234851407
nll: 14.993409339414972
Loss: -24.11556704554221
EPOCH: 2
det bpr: 0.8692492456640573
Pert bpr: 0.8062644345674668
nll: 14.922312380688332
Loss: -24.187933037024003
EPOCH: 3
det bpr: 0.8824175999394804
Pert bpr: 0.8079786282051634
nll: 14.765345924330438
Loss: -24.239358846154904
EPOCH: 4
det bpr: 0.8874757805979973
Pert bpr: 0.8107082456606465
nll: 14.556107902014535
Loss: -24.321247369819396
EPOCH: 5
det bpr: 0.8890512082556646
Pert bpr: 0.8127854277897777
nll: 14.295174962233537
Loss: -24.383562833693333
EPOCH: 6
det bpr: 0.891919630320086
Pert bpr: 0.8152300516925062
nll: 14.06423405610141
Loss: -24.456901550775186
EPOCH: 7
det bpr: 0.8948329354103943
Pert bpr: 0.8173704148594939
nll: 13.886748993050082
Loss: -24.521112445784816
EPOCH: 8
det bpr: 0.9009203799807242
Pert bpr: 0.8198912759081133
nll: 13.73

In [26]:

model = MixtureOfTruncNormModel(num_components=num_components, S=S, low=0, high=150)
if init_idx is not None:
    # Reproducibly, randomly generate some numbers using a numpy rng
    init_rng = np.random.RandomState(1989)
    # generate 20 sets of 2 floats between 0.5 and 60
    all_means = init_rng.uniform(8, 102, (20, num_components))
    # generate 20 sets of 2 floats between 0.25 and 6
    all_scales = init_rng.uniform(0.25, 8, (20, num_components))
    # generate 20 lists length S containing lists length 2 which sum to 1
    all_mix_weights = init_rng.dirichlet([0.5]*num_components, (20,S))

    softinv_means = torch.tensor(all_means[init_idx]) + torch.log(-torch.expm1(torch.tensor(-all_means[init_idx])))
    softinv_scales = torch.tensor(all_scales[init_idx]) - 0.2 + torch.log(-torch.expm1(torch.tensor(-all_scales[init_idx] + 0.2)))
    mix_weights = torch.log(1e-13 + torch.tensor(all_mix_weights[init_idx], dtype=torch.float32))

    # cast to float32
    softinv_means = softinv_means.float()
    softinv_scales = softinv_scales.float()
    mix_weights = mix_weights.float()
    model.update_params(torch.cat([softinv_means, softinv_scales, mix_weights.view(-1)]))

optimizer = torch.optim.Adam(model.parameters(), lr=step_size)
train_y_TS = torch.tensor(train_y_TS, dtype=torch.float32)
M_score_func =  100
M_action = 100
train_T = train_y_TS.shape[0]
from functools import partial
topk_func = partial(top_k_onehot_indicator, k=K)
perturbed_top_K_func = perturbed(topk_func, sigma=perturbed_noise)
losses, bprs, nlls = [], [], []
for epoch in range(100):
    print(f'EPOCH: {epoch}')
    loss, bpr, nll, model = train_epoch(model, optimizer, K, threshold, train_T, M_score_func, M_action, train_y_TS, perturbed_top_K_func, bpr_weight, nll_weight)
    losses.append(loss)
    bprs.append(bpr)
    nlls.append(nll)

/tmp/ipykernel_177719/3122611628.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_y_TS = torch.tensor(train_y_TS, dtype=torch.float32)


EPOCH: 0


/cluster/home/kheuto01/code/prob_diff_topk/torch_training.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  positive_bpr_T = torch_bpr_uncurried(ratio_rating_TS, torch.tensor(train_y_TS), K=K, perturbed_top_K_func=perturbed_top_K_func)
/cluster/home/kheuto01/code/prob_diff_topk/torch_training.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  nll = torch.mean(-mix_model.log_prob( torch.tensor(train_y_TS)))


det bpr: 0.8481752872467041
Pert bpr: 0.8023001551628113
nll: 14.704272270202637
Loss: -24.06900405883789


/cluster/home/kheuto01/code/prob_diff_topk/torch_training.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  deterministic_bpr_T = deterministic_bpr(ratio_rating_TS, torch.tensor(train_y_TS), K=K)


EPOCH: 1
det bpr: 0.8586022257804871
Pert bpr: 0.8042523264884949
nll: 14.994690895080566
Loss: -24.1275691986084
EPOCH: 2
det bpr: 0.8769272565841675
Pert bpr: 0.8064742684364319
nll: 15.254426002502441
Loss: -24.19422721862793
EPOCH: 3
det bpr: 0.8753347396850586
Pert bpr: 0.8080005645751953
nll: 15.113702774047852
Loss: -24.24001693725586
EPOCH: 4
det bpr: 0.8871694803237915
Pert bpr: 0.8108158111572266
nll: 15.023340225219727
Loss: -24.324474334716797
EPOCH: 5
det bpr: 0.890000581741333
Pert bpr: 0.8129664659500122
nll: 15.006230354309082
Loss: -24.388994216918945
EPOCH: 6
det bpr: 0.8919475674629211
Pert bpr: 0.8152533173561096
nll: 15.023977279663086
Loss: -24.457599639892578
EPOCH: 7
det bpr: 0.8947763442993164
Pert bpr: 0.8175246715545654
nll: 14.976813316345215
Loss: -24.525739669799805
EPOCH: 8
det bpr: 0.9001567363739014
Pert bpr: 0.8201390504837036
nll: 14.867987632751465
Loss: -24.604171752929688
EPOCH: 9
det bpr: 0.9156369566917419
Pert bpr: 0.8223062753677368
nll: 14.695

In [16]:
all_means.dtype

dtype('float64')

In [21]:
for param in model.parameters():
    print(param.dtype)

torch.float64
torch.float64
torch.float64


In [23]:
softinv_scales.dtype

torch.float64

In [24]:
mix_weights.dtype

torch.float32